# OPTIMIZING MODEL PARAMETERS
## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters
* **Number of Epochs** - the number times to iterate over the dataset
* **Batch Size** - the number of data samples propagated through the network before the parameters are updated
* **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop
### Each epoch consists of two main parts:
* **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
* **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

## Loss Function

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Inside the training loop, optimization happens in three steps:
* Call ```optimizer.zero-grad()``` to reset the gradients of model parameters. Gradients by default add up, to prevent double-counting, we explicitly zero them at each iteration.
* Backpropagate the prediction loss with a call to ```loss.backward().``` PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call ```optimizer.step()``` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy:  {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t + 1}\n------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
------------------------
loss: 2.312783   [    0/60000]
loss: 2.296212   [ 6400/60000]
loss: 2.276044   [12800/60000]
loss: 2.268178   [19200/60000]
loss: 2.255830   [25600/60000]
loss: 2.228435   [32000/60000]
loss: 2.241695   [38400/60000]
loss: 2.211011   [44800/60000]
loss: 2.219706   [51200/60000]
loss: 2.181313   [57600/60000]
Test Error: 
 Accuracy:  46.7%, Avg loss: 2.174539 

Epoch 2
------------------------
loss: 2.186075   [    0/60000]
loss: 2.176430   [ 6400/60000]
loss: 2.121386   [12800/60000]
loss: 2.140170   [19200/60000]
loss: 2.097557   [25600/60000]
loss: 2.030914   [32000/60000]
loss: 2.073171   [38400/60000]
loss: 1.994948   [44800/60000]
loss: 2.014253   [51200/60000]
loss: 1.941958   [57600/60000]
Test Error: 
 Accuracy:  56.5%, Avg loss: 1.933756 

Epoch 3
------------------------
loss: 1.960412   [    0/60000]
loss: 1.935279   [ 6400/60000]
loss: 1.820058   [12800/60000]
loss: 1.870477   [19200/60000]
loss: 1.764893   [25600/60000]
loss: 1.695623   [32